# Views vs Copies
The concept of views and copies is important to prevent bugs that are difficult to identify and solve in Pandas. The package Pandas, which is the main focus in the course, is built on top of the package Numpy. Therefore, we first explore the copy vs view behavior of Numpy.

#### Views in Numpy

In [1]:
import numpy as np

First, we create a one dimensional Numpy Array that will be used for the demonstration. 

In [2]:
vector = np.array([1, 2, 3, 4, 5])
type(vector)

Like mentioned during the lecture, the numpy array has a certain data buffer. To see the data buffer of a numpy array, you can use the *base* property:

In [3]:
vector.base

The base property returns None when the vector does not have a base vector from which it uses the data buffer. However, the moment we create a view of the initial array, the base vector returns the data buffer of the base:

In [4]:
view = vector.view()
view.base

Since this is a view, if we modify the data (buffer) of the view, this will also change the value of the *vector* array since they share the same buffer.

In [5]:
view[0] = 100
print(view)
print(vector)

Likewise, if we change the original vector, we also change the view:

vector[0] = 1
print(view)
print(vector)

#### Copies in Numpy
Alternatively, we can copy the data instead of creating a view. In that case, both the metadata, and the data buffer are copied to a new object:

In [6]:
copy = vector.copy()
copy.base

As you can see, the copy does not have a base. This is because it is an entirely new object! If we now change a value in the copy, this will not impact the vector:

In [7]:
copy[0] = 100
print(copy)
print(vector)

#### Why important?
Wgen doing data manipulation, you will typically use Numpy functions. Because of memory optimization, these functions try to use views as much as possible; however, sometimes it is not possible to use views and the function will use copies instead. Not realizing that you are working on a copy of the data instead of a view can introduce some nasty bugs!

In [8]:
time_series = np.array([100.10, 105.78, 98.54, 10098, 102.83, 105.98, 101.26])

We work with the time series above. In a first step, we are going to filter the data to include only values that are greater than 100:

In [9]:
greater_100 = time_series[time_series > 100]
greater_100

By doing this, we realize that we have an outlying value that seems to be a mistake in data input. We quickly solve this:

In [10]:
time_series[3] = time_series[3]/100
time_series

Now, everything looks fine and we will continue with computing the average for all values greater than 100:

In [11]:
np.mean(greater_100)

**What happened?** I assumed that the data filtering returned a view, but it returned a copy! The fix I did on the time series array did not result in a fix in the greater_100 array.

In [12]:
greater_100.base

##### Example 2
Let's say we now want to take the average of the first 5 values of the array; however, these are represented in euros and we want to represent the average in cents. Therefore, we first filter the array based on the index:

In [13]:
first_5 = time_series[:5]
first_5

Now that we obtained the filtered array, we multiple each value by 100:

In [14]:
first_5 *= 100
first_5

Next, we compute the mean:

In [15]:
np.mean(first_5)

Now, we continue the project with the original array. But if we are not careful, then we do not notice that the original array now looks like this:

In [16]:
print(time_series)

**What happened?** The subset operation returned a view instead of a copy! The same happened when we updated the values of the array by multipying them by 100. 

In [17]:
first_5.base

**Note:** if we used *first_5 = first_5 \* 100* instead of *first_5 \*= 100*, then we would not have had this issue because this returns a copy of the array!

In [18]:
time_series = np.array([100.10, 105.78, 98.54, 100.98, 102.83, 105.98, 101.26])
first_5 = time_series[:5]
first_5 = first_5 * 100 
print(first_5)
print(time_series)

In [19]:
first_5.base

## Pandas
Now, let's move on to pandas.

In [20]:
import pandas as pd

We use the movies database like the other notebook:

In [21]:
movies = pd.read_parquet('https://kuleuven-mda.s3.eu-central-1.amazonaws.com/movies.parquet.gzip')
movies.release_date = pd.to_datetime(movies.release_date)
movies.id = pd.to_numeric(movies.id)
movies.original_language = pd.Categorical(movies.original_language)
movies.head()

Let's start with subsetting the data. We first select a single column:

In [22]:
vote_counts = movies.loc[:, "vote_count"]
vote_counts._is_view

This returns a view of the data. This makes sense because we are doing the operation in a single block so the Numpy operation can return a view without any issues:

In [23]:
movies._mgr.blocks

Now, let's filter both rows and columns:

In [24]:
vote_counts = movies.loc[1:4, "vote_count"]
vote_counts._is_view

In [25]:
mask = (movies.id == 862) | (movies.id == 8884)
movies.loc[mask, "vote_count"]._is_view

And let's now select multiple columns:

In [26]:
movies.loc[:, ['vote_count', 'vote_average']]._is_view